In [ ]:
import pickle
import rospy
import baxter_interface
import os.path as path
import copy
from tqdm import tqdm_notebook as tqdmn
import matplotlib.pyplot as plt
import itertools
import numpy as np
import time

from collections import namedtuple
import random

# Taken from
# https://github.com/pytorch/tutorials/blob/master/Reinforcement%20(Q-)Learning%20with%20PyTorch.ipynb

Transition = namedtuple('Transition', ('state', 'action', 'mask', 'next_state',
                                       'reward'))


class Memory(object):
    def __init__(self):
        self.memory = []

    def push(self, *args):
        """Saves a transition."""
        self.memory.append(Transition(*args))

    def sample(self, batch_size=None):
        if batch_size is None:
            return Transition(*zip(*self.memory))
        else:
            random_batch = random.sample(self.memory, batch_size)
            return Transition(*zip(*random_batch))

    def append(self, new_memory):
        self.memory += new_memory.memory

    def __len__(self):
        return len(self.memory)

In [ ]:
# initialize ros node
rospy.init_node('lambda_trainer')
limb = baxter_interface.Limb('right')

In [ ]:
# define relevant variables
PLAYBACK_MODE = 'bend'
REACH_TIME = 0.05
file_seed = path.expanduser('~/data/moveit_data/')
if PLAYBACK_MODE == 'bend':
    moveit_file = file_seed + 'bend_dof_'
else:
    file_seed = file_seed + 'full_dof_'
limb.set_joint_position_speed(0.75)

In [ ]:
# Learning related parameters

H_SIZE = (20, 20)
NUM_JOINT = 3
MAX_ITER_NUM = 1000
OPTIM_EPOCHS = 5
OPTIM_BATCH_SIZE = 64
LEARNING_RATE =0.0003
GAMMA = 0.99
TAU = 0.95

In [ ]:
# torch imports
from torch.autograd import Variable
from ppo_algorithm.utils import *

# ppo imports
from ppo_algorithm.models import Policy, Value
from ppo_algorithm.core import ppo_step, estimate_advantages

# class for the PPO model

class PPOPolicy(object):
    def __init__(self, state_dim = NUM_JOINT*4, action_dim = NUM_JOINT, policy_log_std = 0):
        self.policy_net = Policy(state_dim, action_dim, log_std = policy_log_std, hidden_size = H_SIZE)
        self.value_net = Value(state_dim)
        self.max_iter_num = MAX_ITER_NUM
        self.optimizer_policy = torch.optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)
        self.optimizer_value = torch.optim.Adam(value_net.parameters(), lr=LEARNING_RATE)
        
    def update_params(self, batch, i_iter):
        states = torch.from_numpy(np.stack(batch.state))
        actions = torch.from_numpy(np.stack(batch.action))
        rewards = torch.from_numpy(np.stack(batch.reward))
        masks = torch.from_numpy(np.stack(batch.mask).astype(np.float64))
        if use_gpu:
            states, actions, rewards, masks = states.cuda(), actions.cuda(), rewards.cuda(), masks.cuda()
        values = self.value_net(Variable(states, volatile=True)).data
        fixed_log_probs = self.policy_net.get_log_prob(Variable(states, volatile=True), Variable(actions)).data

        """get advantage estimation from the trajectories"""
        advantages, returns = estimate_advantages(rewards, masks, values, GAMMA, TAU, use_gpu)

        lr_mult = max(1.0 - float(i_iter) / self.max_iter_num, 0)

        """perform mini-batch PPO update"""
        optim_iter_num = int(math.ceil(states.shape[0] / OPTIM_BATCH_SIZE))
        for _ in range(OPTIM_EPOCHS):
            perm = torch.randperm(states.shape[0])

            states, actions, returns, advantages, fixed_log_probs = \
                states[perm], actions[perm], returns[perm], advantages[perm], fixed_log_probs[perm]

            for i in range(optim_iter_num):
                ind = slice(i * OPTIM_BATCH_SIZE, min((i + 1) * OPTIM_BATCH_SIZE, states.shape[0]))
                states_b, actions_b, advantages_b, returns_b, fixed_log_probs_b = \
                    states[ind], actions[ind], advantages[ind], returns[ind], fixed_log_probs[ind]

                ppo_step(self.policy_net, self.value_net, self.optimizer_policy, self.optimizer_value, 1, states_b, actions_b, returns_b,
                         advantages_b, fixed_log_probs_b, lr_mult, LEARNING_RATE, CLIP_EPSILON, L2_REG)
    
    def select_action(self, state, mean_flag):
        if mean_flag:
            return self.policy_net(state)[0].data[0].np()
        else:
            return self.policy_net.select_action(state)[0].np()


In [5]:
# in a loop load all the plans and replay
joint_angles = {}
err = []
xaxis = []
# observations = []

def getState(limb, names):
    measured_pos = []
    measured_vel = []
    measured_torque = []
    for name in names:
        measured_pos.append(limb.joint_angles()[name])
        measured_vel.append(limb.joint_velocities()[name])
        measured_torque.append(limb.joint_efforts()[name])
    return np.array(measured_pos), np.array(measured_vel), np.array(measured_torque)

def reward(goal, measured):
    return -(np.sum(np.array(goal) - np.array(measured))**2)

def within_threshold(goal, measured, threshold):
    if np.abs(np.array(goal) - np.array(measured)).all() < threshold:
        return True
    else:
        return False
    
def action_step(initial_setpoint, delta, moving_joints, all_joints):
    joint_angles = dict()
    moving_ind = 0
    for i, joint in enumerate(all_joints):
        joint_angles[joint] = initial_setpoint[i]
        if joint in moving_joints:
            joint_angles[joint] = initial_setpoint[moving_ind] + delta[moving_ind]
            moving_ind += 1
    
    return (joint_angles)
            
    

def generateTrajectory(numSteps = 1000):
    positions = []
    for i in range(numSteps):
        positions.append(np.clip(np.cos(float(i)*(2*np.pi)/1000)*0.5+0.55, -0, 3))
    return positions


# every 1:00min update policy parameters
# after a full file playback, grab a new random file
# full outer loop is max updates



class PPO_Train():
    def __init__(self, policy_net, traj_type, num_files, angle_threshold, moving_joints, train_episodes, 
                 active_joint='right_e1', compensated_joint='right_s1'):
        self.policy = policy_net
        
        #are we using random files or 1dof sine
        self.traj_type = traj_type
        
        #moveit attributes
        self.num_files = num_files
        self.angle_threshold = angle_threshold
        self.moving_joints = moving_joints  #also used for sine
        self.train_episodes = train_episodes
        
        #1dof attributes
        self.active_joint = active_joint
        self.compensated_joint = compensated_joint
        
        #limb object
        self.limb = baxter_interface.Limb('right')
    
    def getState(self, names):
        measured_pos = []
        measured_vel = []
        measured_torque = []
        for name in names:
            measured_pos.append(self.limb.joint_angles()[name])
            measured_vel.append(self.limb.joint_velocities()[name])
            measured_torque.append(self.limb.joint_efforts()[name])
        return np.array(measured_pos), np.array(measured_vel), np.array(measured_torque)

    def reward(self, goal, measured):
        return -(np.sum(np.array(goal) - np.array(measured))**2)

    def reward1d(self, goal, measured):
        return -(np.array(goal) - np.array(measured)**2)    
    
    def within_threshold(self, goal, measured, threshold):
        if np.abs(np.array(goal) - np.array(measured)).all() < threshold:
            return True
        else:
            return False

    def action_step(self, initial_setpoint, delta, moving_joints, all_joints):
        joint_angles = dict()
        moving_ind = 0
        for i, joint in enumerate(all_joints):
            joint_angles[joint] = initial_setpoint[i]
            if joint in moving_joints:
                joint_angles[joint] = initial_setpoint[moving_ind] + delta[moving_ind]
                moving_ind += 1

        return (joint_angles)
    
    def initialAngles(self, angles_dict):
        self.limb.move_to_joint_positions(angles_dict)

    def generateTrajectory(self, numSteps = 1000):
        positions = []
        for i in range(numSteps):
            positions.append(np.clip(np.cos(float(i)*(2*np.pi)/1000)*0.5+0.55, -0, 3))
        return positions    
    
    def train(self):
        if self.traj_type == 'moveit':
            for episode in range(self.train_episodes):
                if trajectory_done:
                    file_number = np.random.randint(self.num_files)
                    plan = pickle.load(open(moveit_file + str(file_number) + '.pkl', 'rb'))
                    goal_list = []
                    for i, pt in enumerate(plan):
                        if i == 0:
                            joint_names = copy.deepcopy(plan[i])
                        else:
                            goal_list.append(pt)
                    trajectory_done = False
                    memory = Memory()

                while goal_list:
                    goal_current_full = goal_list.pop(0)
                    goal_current = []
                    
                    #chop the dimension of the goal to num_moving_joints
                    for i, name in enumerate(self.moving_joints):
                        ind = joint_names.index(name)
                        goal_current.append(goal_current_full[ind])
                        
                    done = False

                    angles, velocities, torques = self.getState(limb, self.moving_joints)
                    s_goal = np.hstack([np.array(goal_current)])
                    s = np.hstack([angles.copy(), velocities.copy(), torques.copy(), s_goal.copy()])
                    s_torch = Variable(torch.from_numpy(s)).unsqueeze(0)

                    start_time = time.time()
                    while((time.time() - start_time) < REACH_TIME):
#                         action = policy.sample_action(s)  #proper torch conversion in and out
                        joint_angles = self.action_step(goal_current_full, action, self.moving_joints, joint_names)
                        self.limb.set_joint_positions(joint_angles)

                    angles_new, velocities_new, torques_new = self.getState(limb, self.moving_joints)
                    s_ = np.hstack([angles_new.copy(), velocities_new.copy(), torques_new.copy(), s_goal.copy()])
                    r = self.reward(goal_current, angles_new)
    #                 if within_threshold(goal_current, angles, angle_threshold):
    #                     done = 1
    #                     memory.push(s, a, r, s_, done)
    #                     break
                    done = 1
                    memory.push(s, a, done, s_, r)
                    s = s_
                return (memory)
                trajectory_done = True
                                        
        elif self.traj_type == 'sine':
            iter_list = generateTrajectory(num_steps=1000)
            goal_list = [self.limb.joint_angles()[compensated_joint]]*len(iter_list)
                                        
            for i, goal_current in enumerate(goal_list):
                done = False

                angles, velocities, torques = self.getState(limb, self.moving_joints)
                s_goal = np.array([iter_list[i], goal_current])
                s = np.hstack([angles.copy(), velocities.copy(), torques.copy(), s_goal.copy()])
                s_torch = Variable(torch.from_numpy(s)).unsqueeze(0)

                start_time = time.time()
                while((time.time() - start_time) < REACH_TIME):
#                     action = policy.sample_action(s)
                    joint_angles = self.limb.joint_angles()
                    joint_angles[active_joint] = iter_list[i]
                    joint_angles[compensated_joint] = goal_current + action
                    self.limb.set_joint_positions(joint_angles)

                angles_new, velocities_new, torques_new = self.getState(limb, self.moving_joints)
                s_ = np.hstack([angles_new.copy(), velocities_new.copy(), torques_new.copy(), s_goal.copy()])
                r = self.reward1d(goal_current, angles_new[self.moving_joints.index(compensated_joint)])
#                 if within_threshold(goal_current, angles, angle_threshold):
#                     done = 1
#                     memory.push(s, a, r, s_, done)
#                     break
                done = 1
                memory.push(s, a, done, s_, r)
                s = s_    
            return (memory)

In [ ]:
trajectory_done = True
num_files = 50
angle_threshold = 0.01

# trajectory_type = 'moveit'
# moving_joints = ['right_s1', 'right_e1', 'right_w1']

trajectory_type = 'sine'
moving_joints = ['right_s1', 'right_e1']
NUM_JOINT=2

EPISODES = 1000
EPOCHS = 10

# PPO = PPOPolicy(state_dim = NUM_JOINT*4, action_dim = NUM_JOINT, policy_log_std = 0)
PPO = PPOPolicy(state_dim = NUM_JOINT*4, action_dim = 1, policy_log_std = 0)

trainer = PPO_Train(policy_net=PPO.policy_net, traj_type=trajectory_type, num_files, angle_threshold, moving_joints, train_episodes=EPISODES, 
                 active_joint='right_e1', compensated_joint='right_s1')

initial_angles = {'right_s0': , 'right_s1': , 'right_e0':, 'right_e1': 'right_w0':, 'right_w1':, 'right_w2': }
trainer.initialAngles(initial_angles)

for i in range(EPOCHS):
    batch = trainer.train()
    PPO.update_params(batch, i)

In [ ]:
# plot commanded trajectory and error
joint_angles = {}
for file_iter in tqdmn(xrange(2), desc='Files read:'):
    plan = pickle.load(open(moveit_file + str(file_iter) + '.pkl', 'rb'))
    s1 = [row[1] for row in plan[1:]]
    e1 = [row[3] for row in plan[1:]]
    w1 = [row[5] for row in plan[1:]]
    plt.figure(file_iter)
    plt.plot(np.array(s1)*180/np.pi, 'b')
    plt.plot(np.array(e1)*180/np.pi, 'r')
    plt.plot(np.array(w1)*180/np.pi, 'g')
    plt.plot(np.array(err[file_iter]['e1'])*180./np.pi, 'b--')
    plt.plot(np.array(err[file_iter]['s1'])*180./np.pi, 'r--')
    plt.plot(np.array(err[file_iter]['w1'])*180./np.pi, 'g--')
    plt.legend(['e1', 's1', 'w1'])
    plt.show()

plt.figure(0)
plt.plot(np.array(err[0]['e1'])*180./np.pi, 'b')
plt.plot(np.array(err[0]['s1'])*180./np.pi, 'r')
plt.plot(np.array(err[0]['w1'])*180./np.pi, 'g')
plt.legend(['e1', 's1', 'w1'])
plt.show()
plt.figure(1)
plt.plot(np.array(err[1]['e1'])*180./np.pi, 'b')
plt.plot(np.array(err[1]['s1'])*180./np.pi, 'r')
plt.plot(np.array(err[1]['w1'])*180./np.pi, 'g')
plt.legend(['e1', 's1', 'w1'])
plt.show()